In [1]:
import os
import torch
from threading import Thread
from transformers import AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, AutoModel

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
MODEL_PATH = os.environ.get('MODEL_PATH', '/data/zhenyu/LLM_Model/GLM_4/GLM4_Chat/')

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)

model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    # attn_implementation="flash_attention_2", # Use Flash Attention
    # torch_dtype=torch.bfloat16, #using flash-attn must use bfloat16 or float16
    device_map="auto").eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: ChatGLMForConditionalGeneration does not support Flash Attention 2.0 yet. Please request to add support where the model is hosted, on its model hub page: https://huggingface.co//data/zhenyu/LLM_Model/GLM_4/GLM4_Chat//discussions/new or in the Transformers GitHub repo: https://github.com/huggingface/transformers/issues/new

In [ ]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(messages):
    stop = StopOnTokens()
    model_inputs = tokenizer.apply_chat_template(messages,
                                                 add_generation_prompt=True,
                                                 tokenize=True,
                                                 return_tensors="pt").to(next(model.parameters()).device)
    streamer = TextIteratorStreamer(tokenizer, timeout=60, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 512,
        "do_sample": False,
        "top_p": 1,
        "temperature": 0,
        "stopping_criteria": StoppingCriteriaList([stop]),
        "repetition_penalty": 1,
        "eos_token_id": model.config.eos_token_id,
    }
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    result = ''
    for new_token in streamer:
        result += new_token
        print(new_token, end='')
    return result


In [ ]:
system_prompt_1 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC > 9, CTAI 结节风险 > 85%：分析解读参考 “ CAC、CTAI均在高风险区间，应选择积极治疗，特别是结节大小≥8mm，手术干预指征更强。如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。当然，尽管CAC大于9，CTAI＞85%，提示肺结节恶性风险高危，但手术切除后病理依然存在良性结节的可能，虽然这种可能性比较小”。
CAC > 9, CTAI 结节风险 < 85%：分析解读参考 “ CAC高风险区间，CTAI未达高风险区间，建议抗炎治疗（两周）后3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗，如果没有结节增大、密度增高、实性成分增加等，应对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。”
```
"""
messages_1 = []
messages_1.append({"role":"system", "content":system_prompt_1})
check_info_1 = """
患者姓名：尹玉梅；
CAC: 20;
结节信息: 
1号结节 结节风险预测：90%；结节类型：混合型；结节大小：15mm, 
2号结节 结节风险预测：80%；结节类型：磨玻璃型；结节大小：6mm；
"""
messages_1.append({"role":"user", "content":check_info_1})
import time
start_time = time.time()
predict(messages_1)
end_time = time.time()
print()
print('Use time: ', end_time-start_time)

In [ ]:
system_prompt_2 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC > 9, CTAI 结节风险 < 85%：分析解读参考 “ CAC高风险区间，CTAI未达高风险区间，建议抗炎治疗（两周）后3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗，如果没有结节增大、密度增高、实性成分增加等，应对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。”
```
"""
messages_2 = []
messages_2.append({"role":"system", "content":system_prompt_2})
check_info_2 = """
患者姓名：刘贵玲；
CAC: 12;
CTAI: 
1号结节 结节风险预测：57%；结节类型：磨玻璃型；结节大小：6.4mm；
"""
messages_2.append({"role":"user", "content":check_info_2})
predict(messages_2)

In [ ]:
system_prompt_3 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC < 3, CTAI 结节风险 < 85%：分析解读参考 “良性结节的可能性较大，建议年度体检，12个月复查胸部CT。如果结节大于15mm，或患者焦虑明显、要求治疗意愿明显，可考虑手术干预，但术后病理良性结节概率较高。”
```
"""
messages_3 = []
messages_3.append({"role":"system", "content":system_prompt_3})
check_info_3 = """
患者姓名：衡凤英；
CAC: 2;
CTAI: 
1号结节 结节风险：56%；结节类型：实性结节；结节大小：8.7mm；
2号结节 结节风险：61%；结节类型：混合型结节；结节大小：5.5mm；
"""
messages_3.append({"role":"user", "content":check_info_3})
predict(messages_3)

In [ ]:
system_prompt_4 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC < 3, CTAI 结节风险 < 85%：分析解读参考 “良性结节的可能性较大，建议年度体检，12个月复查胸部CT。如果结节大于15mm，或患者焦虑明显、要求治疗意愿明显，可考虑手术干预，但术后病理良性结节概率较高。”
```
"""
messages_4 = []
messages_4.append({"role":"system", "content":system_prompt_4})
check_info_4 = """
患者姓名：张学阳；
CAC: 0;
CTAI: 
1号结节 结节风险：0%；结节类型：实性结节；结节大小：8.7mm；
2号结节 结节风险：1%；结节类型：混合型结节；结节大小：9mm；
"""
messages_4.append({"role":"user", "content":check_info_4})
predict(messages_4)

In [ ]:
system_prompt_5 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
 CAC < 3, CTAI 结节风险 > 85%：分析解读参考 “CAC低风险区间，CTAI高风险区间，建议密切随访观察。每3个月复查胸部CT，同时进行CTAI随访对比，如有结节增大、密度增高、实性成分增加等，可建议积极治疗；如果没有结节增大、密度增高、实性成分增加等，应继续对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。”
```
"""
messages_5 = []
messages_5.append({"role":"system", "content":system_prompt_5})
check_info_5 = """
患者姓名：张凤玲；
CAC: 0;
CTAI: 
1号结节 结节风险：89%；结节类型：混合型结节；结节大小：8.7mm；
"""
messages_5.append({"role":"user", "content":check_info_5})
predict(messages_5)

In [ ]:
system_prompt_6 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
3 ≤ CAC ≤ 9, CTAI 结节风险 > 85%：分析解读参考“CAC中高风险区间，CTAI高风险区间，应选择积极治疗。特别是结节大小≥8mm，手术干预指征更强，如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。”
3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%：分析解读参考“CAC中高风险区间，CTAI未达高风险区间，应对患者进行定期随访，3-6个月复查胸部CT，同时CTAI进行随访对比分析。”

```
"""
messages_6 = []
messages_6.append({"role":"system", "content":system_prompt_6})
check_info_6 = """
患者姓名：陈德良；
CAC: 4;
CTAI: 
1号结节 结节风险：93%；结节类型：混合型结节；结节大小：15mm；
2号结节 结节风险：67%；结节类型：实性结节；结节大小：8mm；
"""
messages_6.append({"role":"user", "content":check_info_6})
predict(messages_6)

In [ ]:
system_prompt_7 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%：分析解读参考“CAC中高风险区间，CTAI未达高风险区间，应对患者进行定期随访，3-6个月复查胸部CT，同时CTAI进行随访对比分析。”

```
"""
messages_7 = []
messages_7.append({"role":"system", "content":system_prompt_7})
check_info_7 = """
患者姓名：王改荣；
CAC: 4;
CTAI: 
1号结节 结节风险：77%；结节类型：混合型结节；结节大小：11mm，与2月22相比，风险降低，体积增加13%，密度降低22%，体积倍增时间698天；
"""
messages_7.append({"role":"user", "content":check_info_7})
predict(messages_7)

In [ ]:
system_prompt_8 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    CAC > 9：高风险区间；
    3 ≤ CAC ≤ 9: 中高风险区间；
    CAC < 3: 低风险区间；
结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
CAC > 9, CTAI 结节风险 < 85%：分析解读参考 “ CAC高风险区间，CTAI未达高风险区间，建议抗炎治疗（两周）后3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗，如果没有结节增大、密度增高、实性成分增加等，应对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。”

```
"""
messages_8 = []
messages_8.append({"role":"system", "content":system_prompt_8})
check_info_8 = """
患者姓名：张爱霞；
CAC: 10;
CTAI: 
1号结节 结节风险：70%；结节类型：混合型结节；结节大小：11mm，较3月5日新发；
随访相同结节都为低风险
"""
messages_8.append({"role":"user", "content":check_info_8})
predict(messages_8)

In [ ]:
system_prompt_9 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    3 ≤ CAC ≤ 9: 中高风险区间；

结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
3 ≤ CAC ≤ 9, CTAI 结节风险 > 85%：分析解读参考“CAC中高风险区间，CTAI高风险区间，应选择积极治疗。特别是结节大小≥8mm，手术干预指征更强，如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。”
```
"""
messages_9 = []
messages_9.append({"role":"system", "content":system_prompt_9})
check_info_9 = """
患者姓名：弓淑琴；
CAC: 3;
CTAI: 
1号结节 结节风险：90%；结节类型：磨玻璃结节；结节大小：12mm；
2号结节 结节风险：90%；结节类型：磨玻璃结节；结节大小：8mm
3号结节 结节风险：88%；结节类型：混合型结节；结节大小：15mm
"""
messages_9.append({"role":"user", "content":check_info_9})
predict(messages_9)

In [ ]:
system_prompt_10 = f"""
你是一个圣美生物有限公司一名经验丰富的呼吸科医生。患者已经做过了CAC（循环染色体异常细胞检测）与CTAI（肺部CT扫描与肺结节风险评估）检测。
请先根据患者提供的CAC检测报告与CTAI检测报告，综合两份检测报告和以下参考信息，对每个结节进行分析解读。
最后给出整体建议。【注意，不要使用肯定词汇，例如：立即，必须，应该等词汇。应考虑结节大小给出随访或积极治疗建议。】
```参考信息：
CAC:
    3 ≤ CAC ≤ 9: 中高风险区间；

结节风险预测：
    结节风险预测 > 85%: 高风险结节；
    65% < 结节风险预测 < 85%: 中高风险结节；
    40% < 结节风险预测 < 65%: 中风险结节；
    结节风险预测 < 40%: 低风险结节；
    
3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%：分析解读参考“CAC中高风险区间，CTAI未达高风险区间，应对患者进行定期随访，3-6个月复查胸部CT，同时CTAI进行随访对比分析。”

```
"""
messages_10 = []
messages_10.append({"role":"system", "content":system_prompt_10})
check_info_10 = """
患者姓名：王向阳；
CAC: 5;
CTAI: 
1号结节 结节风险：23%；结节类型：实性结节；结节大小：12mm；
"""
messages_10.append({"role":"user", "content":check_info_10})
predict(messages_10)